# Kaggle - Bag of Words Meets Bags of Popcorn

**This notebook was created following the [tutorial](https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words) from Kaggle. In this notebook, we will clean and construct bag-of-words on IMDB review texts and use a classifier to predict the semtiment labels of the review.**

In [1]:
import os
import pandas as pd
import numpy as np
from IPython.display import display

from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

## Load Labeled Training Data

For Bag-of-words approach, we're going to use labeled training data for supervised learning. Read the labeled training data in `.tsv` file with `pandas.read_csv`.

In [2]:
train = pd.read_csv('data/labeledTrainData.tsv', header=0, delimiter="\t", quoting=3)

Here, `header=0` indicates the first line of the tsv file contains the column names, `delimiter="\t"` means the file uses tap as delimiter, and `quoting=3` tells Python to ignore double quotes.

Take a look at the training data.

In [3]:
print('Dimension of Labeled Training Data: {}.'.format(train.shape))
print('There are {0} samples and {1} variables in the training data.'.format(train.shape[0], train.shape[1]))

Dimension of Labeled Training Data: (25000, 3).
There are 25000 samples and 3 variables in the training data.


In [4]:
display(train.head())

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


Take a closer look at a review entry.

In [5]:
print(train.review[0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

The review text is in HTML. To parse the content of HTML and get the text between HTML tags, we use the package **BeautifulSoup**.

## Data Cleaning and Text Preprocessing

### Removing HTML Markup with The BeautifulSoup Package

Create BeautifulSoup object to parse HTML.

In [6]:
train['review_bs'] = train['review'].apply(lambda x: BeautifulSoup(x, 'html.parser'))

Use `get_text()` method to get the text content between HTML tags. Take the first review entry as an example,

In [7]:
train.review_bs[0].get_text()

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

## Data Cleaning and Text Preprocessing

### Dealing with Punctuation, Numbers and Stopwords: NLTK and regular expressions

Use **`re`** package to remove digits and punctuations. To build a simplified Bag-of-words model, we remove both digits and punctuations.

In [8]:
train['review_letters_only'] = train['review_bs'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x.get_text()))

The reviews are now strings of letters only.

In [9]:
train['review_letters_only'][0]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    m

Convert words into lower case and tonkenize, i.e. split the text into individual words.

In [10]:
train['review_words'] = train['review_letters_only'].apply(lambda x: x.lower().split())

The reviews are now list of word strings.

In [11]:
train['review_words'][0]

['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 've',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again',
 'maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent',
 'moonwalker',
 'is',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'which',
 'i',
 'remember',
 'going',
 'to',
 'see',
 'at',
 'the',
 'cinema',
 'when',
 'it',
 'was',
 'originally',
 'released',
 'some',
 'of',
 'it',
 'has',
 'subtle',
 'messages',
 'about',
 'mj',
 's',
 'feeling',
 'towards',
 'the',
 'press',
 'and',
 'also',
 'the',
 'obvious',
 'message',
 'of',
 'drugs',

Remove **stop words** with **ntlk**.

In [12]:
set_of_stopwords = set(stopwords.words("english"))
train['review_meaningful_words'] = train['review_words'].apply(lambda x: [w for w in x if not w in set_of_stopwords])

In [13]:
num_removed = len(train['review_words'][0]) - len(train['review_meaningful_words'][0])
print('For the first review entry, the number of stop words removed is {0}.'.format(num_removed))

For the first review entry, the number of stop words removed is 218.


We can also stem the words with `PorterStemmer()` and `Lemmatizer()` to consider only word stems. But for this training data, better results was produced without stemming.

In [14]:
# porter_stemmer = PorterStemmer()
# wordnet_lemmatizer = WordNetLemmatizer()

# train['review_stemmed'] = train['review_meaningful_words'].apply(
#     lambda x: [porter_stemmer.stem(w) for w in x])
# train['review_stemmed'] = train['review_cleaned'].apply(
#     lambda x: [wordnet_lemmatizer.lemmatize(w) for w in x])

As the final step, we join the list of words into a single string.

In [15]:
# train['review_cleaned'] = train['review_stemmed'].apply(lambda x: ' '.join(x)) # uncomment if using stemming
train['review_cleaned'] = train['review_meaningful_words'].apply(lambda x: ' '.join(x)) # comment if using stemming

Add `revies_cleaned` as a new column to the training data.

In [16]:
train.drop(['review', 'review_bs', 'review_letters_only', 'review_words', 'review_meaningful_words'], 
           axis=1, inplace=True)
display(train.head())

,id,sentiment,review_cleaned
0,"""5814_8""",1,stuff going moment mj started listening music ...
1,"""2381_9""",1,classic war worlds timothy hines entertaining ...
2,"""7759_3""",0,film starts manager nicholas bell giving welco...
3,"""3630_4""",0,must assumed praised film greatest filmed oper...
4,"""9495_8""",1,superbly trashy wondrously unpretentious explo...


In [17]:
print(train['review_cleaned'][0])

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

Now the training data is ready for using **Bag of Words**.

## Creating Features from a Bag of Words (Using scikit-learn)

Initialize the `CountVectorizer` object, which is scikit-learn's bag of words tool. `CountVectorizer` converts a collection  of text documents to a matrix of token counts.

In [18]:
vectorizer = CountVectorizer(analyzer="word", preprocessor=None, tokenizer=None, stop_words=None, max_features=5000)

`analyzer="word"` indicates the feature we are using are words; `preprocessor=None`, `tokenizer=None` and `stop_words=None` mean the data needes no more preprocessing, tokenization and removing stop sords since we've already done these in the Data Cleaning and Text Processing step; `max_features=5000` means we only take the top 5000 frequent words as our words in the bag thus limiting the size of the feature vector and speeding up the modeling process.

`fit_transform()` method does two functions: First, it fits the model and learns the vocabulary; second, it transforms our training data into feature vectors. The input to fit_transform should be a list of strings. 

In [19]:
train_data_features = vectorizer.fit_transform(list(train['review_cleaned'].values))

# Numpy arrays are easy to work with, so convert the result to an array
train_data_features = train_data_features.toarray()

The resulting `train_data_features` is a array which contains the occurrence of bag of words in our training data. Take a look at the first row.

In [20]:
train_data_features[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

The `train_data_feature` should be an array with 25000 rows and 5000 columns. Let's check it out,

In [21]:
print('The dimension of train_data_features is {}.'.format(train_data_features.shape))

The dimension of train_data_features is (25000, 5000).


With `get_feature_names()` method, we can take a look at the bag of words which is a list of 5000 words.

In [22]:
vocab = vectorizer.get_feature_names()
print(vocab)

['abandoned', 'abc', 'abilities', 'ability', 'able', 'abraham', 'absence', 'absent', 'absolute', 'absolutely', 'absurd', 'abuse', 'abusive', 'abysmal', 'academy', 'accent', 'accents', 'accept', 'acceptable', 'accepted', 'access', 'accident', 'accidentally', 'accompanied', 'accomplished', 'according', 'account', 'accuracy', 'accurate', 'accused', 'achieve', 'achieved', 'achievement', 'acid', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'activities', 'actor', 'actors', 'actress', 'actresses', 'acts', 'actual', 'actually', 'ad', 'adam', 'adams', 'adaptation', 'adaptations', 'adapted', 'add', 'added', 'adding', 'addition', 'adds', 'adequate', 'admire', 'admit', 'admittedly', 'adorable', 'adult', 'adults', 'advance', 'advanced', 'advantage', 'adventure', 'adventures', 'advertising', 'advice', 'advise', 'affair', 'affect', 'affected', 'afford', 'aforementioned', 'afraid', 'africa', 'african', 'afternoon', 'afterwards', 'age', 'aged', 'agent', 'agents', 'ages', 'aging', 'ago', 'ag

## Modularizing the Cleaning Process

In [23]:
def clean_reviews(reviews, remove_stopwords=False, stem=False):
    """
    to clean review strings
    review: a list of review strings
    remove_stop_words: whether to remove stop words
    output: a list of clean reviews
    """
    # 1. Remove HTML
    reviews_text = list(map(lambda x: BeautifulSoup(x, 'html.parser').get_text(), reviews))
    #
    # 2. Remove non-letters
    reviews_text = list(map(lambda x: re.sub("[^a-zA-Z]"," ", x), reviews_text))
    #
    # 3. Convert words to lower case and split them
    words = list(map(lambda x: x.lower().split(), reviews_text))
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        set_of_stopwords = set(stopwords.words("english"))
        meaningful_words = list(map(lambda x: [w for w in x if not w in set_of_stopwords], words))
    
    # 5. Optionally stem the words
    if stem:
        porter_stemmer = PorterStemmer()
        wordnet_lemmatizer = WordNetLemmatizer()
        stemmed_words = list(map(lambda x: [porter_stemmer.stem(w) for w in x], meaningful_words))
        stemmed_words = list(map(lambda x:[wordnet_lemmatizer.lemmatize(w) for w in x], stemmed_words))
    
        # 6. Join the words to a single string
        clean_review = map(lambda x: ' '.join(x), stemmed_words)
    else:
        clean_review = list(map(lambda x: ' '.join(x), meaningful_words))
    
    return clean_review

## Load and Clean Test Data

In [26]:
# Read the test data
test = pd.read_csv('data/testData.tsv', header=0, delimiter='\t', quoting=3)

# Verify that there are 25,000 rows and 2 columns
print('The dimension of test data is {}.'.format(test.shape))

# Get a bag of words for the test set, and convert to a numpy array
clean_test_reviews = clean_reviews(list(test['review'].values), remove_stopwords=True)
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

The dimension of test data is (25000, 2).


## Random Forest

Let's try a random forest with the features we just created.

In [27]:
# Initialize a Random Forest classifier with 100 trees
rf_clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0) 

# Use cross validation to evaluate the performance of Random Forest
rf_clf_error = 1 - cross_val_score(rf_clf, train_data_features, train['sentiment'], 
                                   cv=5, scoring='accuracy', n_jobs=-1).mean()
print('Random Forest training error: {:.4}'.format(rf_clf_error))

Random Forest training error: 0.1573


## XGBoost

Let's try a XGBoost with the features we created.

In [28]:
# Create xgb trianing set and parameters
dtrain = xgb.DMatrix(train_data_features, label=train['sentiment'])
params = {'silent': 1, 'nthread': -1, 'eval_metric': 'error'}

# Use cross validation to evaluate the performance of XGBoost
print('The cross validation may take a while...')
xgb_cv_results = xgb.cv(params, dtrain, num_boost_round=100, nfold=5, show_stdv=False, seed=0)
xgb_error = xgb_cv_results['test-error-mean'].mean()
print('XGBoost trianing error: {:.4}'.format(xgb_error))

The cross validation may take a while...
XGBoost trianing error: 0.1829


It seems that Random Forest out-performed XGBoost. Thus we'll create a submission file with Random Forest.

### Creating a Submission of Random Forest

In [29]:
# Fit the forest to the training set, using the bag of words as features and the sentiment labels as labels
# This may take a few minutes to run
rf_clf.fit(train_data_features, train['sentiment'])

# Use the random forest to make sentiment label predictions
result = rf_clf.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column an a "sentiment" column
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})

# Use pandas to write the comma-separated output file
output.to_csv("submissions/Bag_of_Words_rf_clf_results.csv", index=False, quoting=3)   
print("Wrote results to submissions/Bag_of_Words_rf_clf_results.csv")

Wrote results to submissions/Bag_of_Words_rf_clf_results.csv


This Bag_of_words with Random Forest model produced a score of 0.84628 without stemming and 0.84476 with stemming.